In [23]:
import numpy as np 																														# type: ignore

In [24]:
def f(x):
	f_0 = 1
	envelope = lambda x : np.exp(-x)  																									# noqa: E731
	return np.sin(x * np.pi * 2 * f_0) * envelope(x)

In [25]:
def ACF(f, W, t, lag):
	return np.sum(
		f[t : t + W] *
		f[lag + t : lag + t +W]
	)

In [26]:
def detect_pitch(f, W, t, sample_rate, bounds):
	ACF_vals = [ACF(f,W,t,i) for i in range(*bounds)]
	sample = np.argmax(ACF_vals) + bounds[0]
	return sample_rate/ sample

In [27]:
sample_rate = 500
start = 0
end = 5
num_samples = int(sample_rate * (end - start) + 1)
window_size = 200
bounds = [20, num_samples // 2]

x = np.linspace(start, end, num_samples)
print(detect_pitch(f(x), window_size, 1, sample_rate, bounds))

25.0


In [28]:
def DF(f, W, t, lag):
	return ACF(f, W, t, 0)\
		+ ACF(f, W, t + lag, 0)\
		- (2 * ACF(f, W, t, lag))

In [30]:
def df_detect_pitch(f, W, t, sample_rate, bounds):
	DF_vals = [DF(f, W, t, i) for i in range(*bounds)]
	sample = np.argmin(DF_vals) + bounds[0]
	return sample_rate/sample

In [29]:
def CMNDF(f, W, t, lag):
	if lag == 0:
		return 1
	return DF(f, W, t, lag)/ np.sum([DF(f, W, t, j + 1) for j in range(lag)]) * lag

In [32]:
def cmndf_detect_pitch(f, W, t, sample_rate, bounds, thresh = 0.1):
	CMNDF_vals = [CMNDF(f, W, t, i) for i in range(*bounds)]
	sample = np.argmin(CMNDF_vals) + bounds[0]

	if sample > thresh:
		sample = 0
		pass

	return sample_rate / sample

In [ ]:
def cmndf_detect_pitch_no_threshold(f, W, t, sample_rate, bounds):
	CMNDF_vals = [CMNDF(f, W, t, i) for i in range(*bounds)]
	sample = np.argmin(CMNDF_vals) + bounds[0]
	return sample_rate / sample

In [ ]:
#sample_rate, data = scipy.io.wavfile.read("singer.wav")
#data = data.astype(np.float64)
#window_size = int (5 / 2000 * sample_rate)
#bounds = [20, 2000]
#data = data[:,0]
#plt.plot(data)

#pitches = []
#for i in range(data.shape[0] // (window_size + 3)):
#	print(i)
#	pitches.append(
#		cmndf_detect_pitch(
#			data,
#			window_size,
#			i * window_size,
#			sample_rate,
#			bounds
#		)
#	)
#plt.plot(pitches)
#print(pitches)
#print(np.max(pitches))
#plt.show